In [5]:
%load_ext autoreload
%autoreload 2

import sys
from datetime import datetime
import time
import random

import ipywidgets as widgets

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from checkers_load import makeBatches, load_games, dataGenerator, getData
import checkers_models
from checkers_display import SimpleSquareBoardDisplay, MoveHeatmapDisplay

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
games = load_games()
# games = games[:30]

X, Y = getData(games)

Size of input is: 1521.05984 MB.


In [3]:
b = SimpleSquareBoardDisplay()
display(b.fig)

Figure(axes=[Axis(label='X', scale=LinearScale(allow_padding=False, max=8.0, min=0.0)), Axis(label='Y', orient…

In [4]:
# Convert from (n,8,4,1)
#         to   (n,8,8,1) with empty squares.
# Should make convolution work better.

n = X.shape[0]

X2 = np.zeros((n,8,8,1))
#print(m.shape, m[::2, ::2].shape, m[1::2, 1::2].shape, x0[::2, :].shape, x0[1::2, :].shape)
X2[:, ::2, 1::2] = X[:, ::2, :]
X2[:, 1::2, ::2] = X[:, 1::2, :]

In [37]:
# Test different ones
for i in range(1000):
    a = (X2[i] if i % 2 == 0 else -X2[i])
    b.update(a)
    time.sleep(1)

KeyboardInterrupt: 

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X2, Y, test_size=0.10, random_state=42)

In [6]:
filters = 128
layers = 0
model = checkers_models.stupidGraphModel(layers=1)

NameError: name 'adj' is not defined

In [ ]:
m_name = "goConvNet-wfilters" + str(filters) + "-wlayers" + str(layers)


log_dir = 'logs/scalars/' + m_name + "-" + datetime.now().strftime("%m%d-%H%M%S")
tf_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
print("!!!!!!!!!!!!! Logging to", log_dir, "!!!!!!!!!!!!!!!!!!")

batch_s = 256
epochs = 20

model.fit(X_train, Y_train,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.1,
          callbacks=[tf_callback])

# Save
f_name = m_name + '.h5'
print("Saving", f_name)
model.save(f_name)

W1012 02:25:37.672106 47284529838080 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1012 02:25:37.716242 47284529838080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:984: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



!!!!!!!!!!!!! Logging to logs/scalars/goConvNet-wfilters128-wlayers0-1012-022535 !!!!!!!!!!!!!!!!!!
Train on 962545 samples, validate on 106950 samples


W1012 02:25:40.026739 47284529838080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:804: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1012 02:25:40.028324 47284529838080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:807: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/20
962545/962545 [==============================] - 33s 35us/step - loss: 5.4199 - acc: 0.0602 - categorical_accuracy: 0.0602 - top3Accuracy: 0.1577 - top5Accuracy: 0.1577 - val_loss: 3.9703 - val_acc: 0.0957 - val_categorical_accuracy: 0.0957 - val_top3Accuracy: 0.2447 - val_top5Accuracy: 0.2447
Epoch 2/20
962545/962545 [==============================] - 33s 35us/step - loss: 3.5571 - acc: 0.1367 - categorical_accuracy: 0.1367 - top3Accuracy: 0.3252 - top5Accuracy: 0.3252 - val_loss: 3.2067 - val_acc: 0.1771 - val_categorical_accuracy: 0.1771 - val_top3Accuracy: 0.4073 - val_top5Accuracy: 0.4073
Epoch 3/20
962545/962545 [==============================] - 33s 35us/step - loss: 2.9451 - acc: 0.2101 - categorical_accuracy: 0.2101 - top3Accuracy: 0.4644 - top5Accuracy: 0.4644 - val_loss: 2.7530 - val_acc: 0.2395 - val_categorical_accuracy: 0.2395 - val_top3Accuracy: 0.5133 - val_top5Accuracy: 0.5133
Epoch 4/20
962545/962545 [==============================] - 33s 35us/step - loss: 

In [97]:
#model = keras.models.load_model(f_name)

figs = []

board_d = SimpleSquareBoardDisplay()
figs.append(board_d.fig)

move_d = MoveHeatmapDisplay()
figs.append(move_d.fig)

pred_d = MoveHeatmapDisplay()
figs.append(pred_d.fig)

display(widgets.HBox(figs))

ValueError: Unknown metric function:top3Accuracy

In [96]:
ix = random.randrange(0,10000)

#for ix in range(30):
x_i, y_i = X_test[ix], Y_test[ix]

x = np.expand_dims(x_i, axis=0)
y_pred = model.predict([x])[0]

#ymax = np.amax(y_pred)
#y_pred = y_pred / ymax


board_d.update(x_i)
move_d.update(y_i)
pred_d.update(y_pred)